In [2]:
import os
if not os.path.exists('templates'):
    os.mkdir('templates')
    
if not os.path.exists('static'):
    os.mkdir('static')

In [12]:
%%writefile templates/home.html

<!DOCTYPE html>
<html>
<head>
	<title>Home</title>
	<!-- <link rel="stylesheet" type="text/css" href="../static/css/styles.css"> -->
	<link rel="stylesheet" type="text/css" href="{{ url_for('static', filename='css/styles.css') }}">
</head>
<body>

	<header>
		<div class="container">
		<div id="brandname">
			An app built with Flask
		</div>
		<h2>Classification</h2>
		
	</div>
	</header>

	<div class="ml-container">

		<form action="{{ url_for('SimilarUtterances')}}" method="POST">
		<p>Enter the Input here</p>
		<!-- <input type="text" name="comment"/> -->
		<textarea name="message" rows="4" cols="50"></textarea>
		<br/>

		<input type="submit" class="btn-info" value="SimilarUtterances">
		
	</form>
		
	</div>

	
	

</body>
</html>

Overwriting templates/home.html


In [4]:
%%writefile templates/result.html

<!DOCTYPE html>
<html>
<head>
	<title></title>
    <link rel="stylesheet" type="text/css" href="{{ url_for('static', filename='css/styles.css') }}">
</head>
<body>

	<header>
		<div class="container">
		<div id="brandname">
			ML App
		</div>
		<h2>Similar Intents</h2>
		
	</div>
	</header>
	<div class="results">


	<h3 style="color:black;">{{prediction}}</h3>

	</div>

</body>
</html>

Writing templates/result.html


In [8]:
%%writefile static/styles.css
body{
	font:15px/1.5 Arial, Helvetica,sans-serif;
	padding: 0px;
	background-color:#f4f3f3;
}

.container{
	width:100%;
	margin: auto;
	overflow: hidden;
}

header{
	background:#03A9F4;#35434a;
	border-bottom:#448AFF 3px solid;
	height:120px;
	width:100%;
	padding-top:30px;

}

.main-header{
			text-align:center;
			background-color: blue;
			height:100px;
			width:100%;
			margin:0px;
		}
#brandname{
	float:left;
	font-size:30px;
	color: #fff;
	margin: 10px;
}

header h2{
	text-align:center;
	color:#fff;

}



.btn-info {background-color: #2196F3;
	height:40px;
	width:100px;} /* Blue */
.btn-info:hover {background: #0b7dda;}


.resultss{
	border-radius: 15px 50px;
    background: #345fe4;
    padding: 20px; 
    width: 200px;
    height: 150px;
}

Overwriting static/styles.css


In [11]:
%%writefile app.py


import json
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances

import pandas as pd
# import tez
# import torch
# import torch.nn as nn
# import transformers
# from sklearn import metrics, model_selection, preprocessing
# from transformers import AdamW, get_linear_schedule_with_warmup
# from sklearn.utils import shuffle
# import torch.nn.functional as nnf
import json
import os
from flask import Flask, request, jsonify, escape, url_for
import requests
# from waitress import serve
from collections import Counter
import logging
import datetime as dt
import time
import math
import re
import string
import socket
import jieba
import enum

DATA_DIR = './'

def spl_char_remove(text):
    split_string=text.split(" ")
    total=[]
    for i in range(0,len(split_string)):
        no=''.join([*filter(str.isalnum,split_string[i])])
        total.append(str(no.strip()))
    listToStr = ' '.join([str(elem) for elem in total]) 
    return listToStr
#     print(listToStr)
spl_char_remove("jadjk12 jdakj %$ sajjh")

def load_df(json_path='name.json'):
    """
    source: borrowed to kaggle competition gstore
    """
    df = pd.read_json(DATA_DIR+json_path)
    
    for column in ['Issues']:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [str(column+"_"+subcolumn) for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    ## function allows to keep the index if we need to merge on the orginal data.
    df = pd.DataFrame([dict(y, index=i) for i, x in enumerate(df['Issues_Messages'].values.tolist()) for y in x])
    
    #print(df.shape)
    return df

def splitDataFrameList(df,target_column,separator):
    
    ''' 
    source: https://gist.github.com/jlln/338b4b0b55bd6984f883 modified to keep punctuation
    df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def split_text(line, separator):
        splited_line =  [e+d for e in line.split(separator) if e]
        return splited_line
    
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df


class Autocompleter:
    def __init__(self):
        pass
    def import_json(self, json_filename):
        #print("load json file...")
        df = load_df(json_filename)
        return df
        
    def process_data(self, new_df):
        #print("select representative threads...")
        new_df = new_df[new_df.IsFromCustomer==False]
        
        #print("split sentenses on punctuation...")
        for sep in ['. ',', ','? ', '! ', '; ']:
            new_df = splitDataFrameList(new_df, 'intent', sep)
            
        #print("Text Cleaning using simple regex...")
        new_df['intent']=new_df['intent'].apply(lambda x: " ".join(x.split()))
        new_df['intent']=new_df['intent'].apply(lambda x: x.strip("."))
        new_df['intent']=new_df['intent'].apply(lambda x: " ".join(x.split()))
        new_df['intent']=new_df['intent'].apply(lambda x: x.replace(' i ',' I '))
        new_df['intent']=new_df['intent'].apply(lambda x: x.replace(' ?','?'))
        new_df['intent']=new_df['intent'].apply(lambda x: x.replace(' !','!'))
        new_df['intent']=new_df['intent'].apply(lambda x: x.replace(' .','.'))
        new_df['intent']=new_df['intent'].apply(lambda x: x.replace('OK','Ok'))
#         new_df['intent']=new_df['intent'].apply(lambda x: x[0].upper()+x[1:])
        new_df['intent']=new_df['intent'].apply(lambda x: x+"?" if re.search(r'^(Wh|How).+([^?])$',x) else x)
        
        print("calculate nb words of sentenses...")
        new_df['nb_words'] = new_df['intent'].apply(lambda x: len(str(x).split(' ')))
        new_df = new_df[new_df['nb_words']>2]
        
        print("count occurence of sentenses...")
        new_df['Counts'] = new_df.groupby(['intent'])['intent'].transform('count')
        
        print("remove duplicates (keep last)...")
        new_df = new_df.drop_duplicates(subset=['intent'], keep='last')
        
        new_df = new_df.reset_index(drop=True)
        print(new_df.shape)  
        
        return new_df
    
    def calc_matrice(self, df):
        # define tfidf parameter in order to count/vectorize the description vector and then normalize it.
        model_tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 10), min_df=0)
        tfidf_matrice = model_tf.fit_transform(df['intent'])
        #print("tfidf_matrice ", tfidf_matrice.shape)
        return model_tf, tfidf_matrice
    def generate_completions(self, prefix_string, data, model_tf, tfidf_matrice):
        
        prefix_string = str(prefix_string)
        new_df = data.reset_index(drop=True)
        weights = new_df['Counts'].apply(lambda x: 1+ np.log1p(x)).values
        # tranform the string using the tfidf model
        tfidf_matrice_spelling = model_tf.transform([prefix_string])
        # calculate cosine_matrix
        cosine_similarite = linear_kernel(tfidf_matrice, tfidf_matrice_spelling)
        
        #sort by order of similarity from 1 to 0:
        similarity_scores = list(enumerate(cosine_similarite))
        similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
        similarity_scores = similarity_scores[0:100]
        similarity_scores = [i for i in similarity_scores]
        similarity_indices = [i[0] for i in similarity_scores]
        #add weight to the potential results that had high frequency in orig data
        for i in range(len(similarity_scores)):
            similarity_scores[i][1][0]=similarity_scores[i][1][0]*weights[similarity_indices][i]
        similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
        similarity_scores = similarity_scores[0:3] # selecting top 3 intnets
        similarity_indices_w = [i[0] for i in similarity_scores]
        
        return new_df.loc[similarity_indices_w]['intent'].tolist(),similarity_scores
import re
def space_remove(text):
    text=re.sub(' +', ' ',text)
    return text

autocompl = Autocompleter()
df1=pd.read_excel("acoe.xlsx")
df1.rename(columns={"intent": "intent1"},inplace=True)
df1["intent"]=df1.intent1.str.lower()
df1["intent"]=df1["intent"].str.strip()

import re
def clean_text(text):
#     text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^A-Za-z0-9]',' ',text)
    return text

df1["intent"]=df1["intent"].apply(lambda x: clean_text(x))
df1["intent"]=df1["intent"].apply(spl_char_remove)
df1["intent"]=df1["intent"].apply(space_remove)
df1["intent"]=df1["intent"].str.strip()

df1.to_excel("acoes.xlsx")
# print(df1.head(5))
# print(df1["Text"][0])
df1=pd.read_excel('acoes.xlsx')
df=df1[["intent","IsFromCustomer"]]
df["intent"]=df["intent"].astype(str).str.strip()
new_df = autocompl.process_data(df)
model_tf, tfidf_matrice = autocompl.calc_matrice(new_df)
dftxt=df.intent.values

def SimUtter(prefix):
    prefix=str(prefix).lower()
    #print(prefix)
    similar_uttrances,utterance_similarity_score=autocompl.generate_completions(prefix, new_df, model_tf,tfidf_matrice)
    return similar_uttrances,utterance_similarity_score

def logit(logs):
    date_today = dt.date.today().strftime("%Y-%m-%d")
    #logging.basicConfig(filename= BASE_PATH + 'hiri/logs/intentProcessor-hiri-'+ str(date_today)+ '.log', level=logging.INFO)
    #logging.info(format(dt.datetime.now()) +" "+ logs)
    

import os
from flask import Flask, render_template, request
from flask import Flask
from string import Template

app = Flask(__name__)

@app.route('/')
def home():
	return render_template('home.html')
    
@app.route('/similarutterances', methods = ['POST'])
def SimilarUtterances():
    #data = request.get_json()
    data=request.form['message']
    
#     for handler in logging.root.handlers[:]:
#         logging.root.removeHandler(handler)

    logit("request received - " + str(data))
    prefix = data#data["prefix"]
    print(prefix)

    similar_uttrances,utterance_similarity_score=SimUtter(prefix)
    final_json = {"similarutterancelist": [
                {"similar_intents": similar_uttrances,
                 "utterance_similarity_score":utterance_similarity_score,
                  }]}

#     jsonData=json.dumps(final_json)
#     logit("after predictions - " + str(jsonData))
#     response = app.response_class(
#     response=jsonData,
#     status=200,
#     mimetype='application/json'
#     )
#     logit("response sent - " + str(response))
    
    return render_template('result.html',prediction = final_json)

if __name__ == "__main__":
	#decide what port to run the app in
	port = int(os.environ.get('PORT', 5000))
	#run the app locally on the givn port
	app.run(host='0.0.0.0', port=port)
	#optional if we want to run in debugging mode
	#app.run(debug=True)

Overwriting app.py


In [10]:
!python app.py

app.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["intent"]=df["intent"].astype(str).str.strip()
calculate nb words of sentenses...
count occurence of sentenses...
remove duplicates (keep last)...
(1246, 4)
 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Mar/2021 10:17:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2021 10:17:48] "GET /static/css/styles.css HTTP/1.1" 404 -
aldhfslkdhflshdf
127.0.0.1 - - [19/Mar/2021 10:17:58] "POST /similarutterances HTTP/1.1" 200 -
127.0.0.1 - - [19/Mar/2021 10:17:58] "GET /static/css/styles.css HTTP/1.1" 404 -
^C
